In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from source.auxiliary_functions import *
from source.real_experiment import clean_abalone

Hamming distance is the relative counting error divided by the number of actually contaminated cells.
Missed is the number of remaining contaminated cells after filtration

# Mistakes according to contamination

Note that the mask returned by the contamination gives 1 if a cell is contaminated and 0 otherwise.
The perturbation removal procedures, on the other hand, give 1 if the cell needs to be kept and 0 otherwise.


In [4]:
def find_stats_perturbation(X, mask, thresh):
    p, n = mask.shape[0], mask.shape[1]
    
    thresh_mask, delta_star = remove_perturbation(X, threshold_ratio=thresh)
    delta = np.sum(thresh_mask*(1-mask))/np.sum(1-mask) # proportion of true data kept
    epsilon = np.sum(thresh_mask*mask)/np.sum(mask) # proportion of contaminated data kept
    return thresh_mask, delta, epsilon

def acc(A,B):
    return np.sum(B*(1-A))/np.sum(B)
    
def hamming(A,B):
    return np.sum(A != (1-B))/A.shape[0]/A.shape[1]

nrep = 20
epsilons = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
#epsilons=[0.01, 0.3]
e_rank = 5
p = 100
n = 1000

### formatting

In [3]:
WIDTH = 470

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8
}

plt.rcParams.update(tex_fonts)

def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

## precision tables

In [10]:
nrep = 10
epsilons = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
#epsilons=[0.01, 0.3]
e_rank = 5
p = 100
n = 1000

def thresholding_experiments(option, intensity):
    # hamming distance between the masks
    hamming_thresh = np.zeros((len(epsilons), nrep))
    hamming_ddc = np.zeros((len(epsilons), nrep))
    hamming_ddc_95 = np.zeros((len(epsilons), nrep))
    hamming_ddc_90 = np.zeros((len(epsilons), nrep))

    #accuracy of the masks
    acc_thresh = np.zeros((len(epsilons), nrep))
    acc_ddc = np.zeros((len(epsilons), nrep))
    acc_ddc_95 = np.zeros((len(epsilons), nrep))
    acc_ddc_90 = np.zeros((len(epsilons), nrep))

    # estimated delta
    delta_thresh = np.zeros((len(epsilons), nrep))
    delta_ddc = np.zeros((len(epsilons), nrep))
    delta_ddc_95 = np.zeros((len(epsilons), nrep))
    delta_ddc_90 = np.zeros((len(epsilons), nrep))
    epsilon_thresh = np.zeros((len(epsilons), nrep))
    epsilon_ddc = np.zeros((len(epsilons), nrep))
    epsilon_ddc_95 = np.zeros((len(epsilons), nrep))
    epsilon_ddc_90 = np.zeros((len(epsilons), nrep))
    
    # spectral error of covariance
    true_error = np.zeros((len(epsilons), nrep))
    error_thresh = np.zeros((len(epsilons), nrep))
    error_ddc = np.zeros((len(epsilons), nrep))
    error_ddc_95 = np.zeros((len(epsilons), nrep))
    error_ddc_90 = np.zeros((len(epsilons), nrep))

    for i,epsilon in enumerate(tqdm(epsilons)):
        for k in range(nrep):
            # generate data
            sigma_true, _ = low_rank(e_rank, p)
            true_norm = np.linalg.norm(sigma_true, ord=2)
            X = np.random.multivariate_normal(np.zeros(p), sigma_true, size=n)

            X_noisy, true_mask = contaminate_bernoulli(X, epsilon, intensity=intensity, option=option)
            
            # compute true error
            true_error[i,k] = (np.linalg.norm(np.cov(X_noisy.T) - sigma_true, ord=2)/true_norm)*100

            # find outliers
            # 1. find_perturbation
            thresh_mask, delta_thresh[i,k], epsilon_thresh[i,k] = find_stats_perturbation(X_noisy, true_mask, 3)
            delta_thresh[i,k] *= (1-epsilon) * 100
            epsilon_thresh[i,k] *= epsilon * 100
            
            # 2. find_outliers_DDC
            ddc_mask, _ = remove_outliers_DDC(X_noisy, 0.99)
            delta_ddc[i,k] = np.sum(ddc_mask*(1-true_mask))/np.sum(1-true_mask) * (1-epsilon) * 100
            epsilon_ddc[i,k] = np.sum(ddc_mask*true_mask)/np.sum(true_mask) * epsilon * 100
            
            # 3. find outliers DDC but with higher tolerance
            ddc_mask_95, _ = remove_outliers_DDC(X_noisy, 0.95)
            delta_ddc_95[i,k] = np.sum(ddc_mask_95*(1-true_mask))/np.sum(1-true_mask) * (1-epsilon) * 100
            epsilon_ddc_95[i,k] = np.sum(ddc_mask_95*true_mask)/np.sum(true_mask) * epsilon * 100
            
            # 4. find outliers DDC but with much higher tolerance
            ddc_mask_90, _ = remove_outliers_DDC(X_noisy, 0.9)
            delta_ddc_90[i,k] = np.sum(ddc_mask_90*(1-true_mask))/np.sum(1-true_mask) * (1-epsilon) * 100
            epsilon_ddc_90[i,k] = np.sum(ddc_mask_90*true_mask)/np.sum(true_mask) * epsilon * 100
            
            # 5. find estimation error under those filtration methods
            sigma_thresh = apply_estimator('oracleMV', X_noisy, mask=1-thresh_mask)
            error_thresh[i,k] = np.linalg.norm(sigma_true - sigma_thresh, ord=2)/true_norm
            sigma_ddc = apply_estimator('oracleMV', X_noisy, mask = 1-ddc_mask)
            error_ddc[i,k] = np.linalg.norm(sigma_true - sigma_ddc, ord=2)/true_norm
            sigma_ddc_95 = apply_estimator('oracleMV', X_noisy, mask=1-ddc_mask_95)
            error_ddc_95[i,k] = np.linalg.norm(sigma_true - sigma_ddc_95, ord=2)/true_norm
            sigma_ddc_90 = apply_estimator('oracleMV', X_noisy, mask=1-ddc_mask_90)
            error_ddc_90[i,k] = np.linalg.norm(sigma_true - sigma_ddc_90, ord=2)/true_norm

    thresh_delta_m = np.mean(delta_thresh, axis=-1)
    thresh_delta_s = np.std(delta_thresh, axis=-1) 
    ddc_delta_m = np.mean(delta_ddc, axis=-1)
    ddc_delta_s = np.std(delta_ddc, axis=-1)
    ddc_delta_95_m = np.mean(delta_ddc_95, axis=-1)
    ddc_delta_95_s = np.std(delta_ddc_95, axis=-1)
    ddc_delta_90_m = np.mean(delta_ddc_90, axis=-1)
    ddc_delta_90_s = np.std(delta_ddc_90, axis=-1)

    thresh_eps_m = np.mean(epsilon_thresh, axis=-1)
    thresh_eps_s = np.std(epsilon_thresh, axis=-1)
    ddc_eps_m = np.mean(epsilon_ddc, axis=-1)
    ddc_eps_s = np.std(epsilon_ddc, axis=-1)
    ddc_eps_95_m = np.mean(epsilon_ddc_95, axis=-1)
    ddc_eps_95_s = np.std(epsilon_ddc_95, axis=-1)
    ddc_eps_90_m = np.mean(epsilon_ddc_90, axis=-1)
    ddc_eps_90_s = np.std(epsilon_ddc_90, axis=-1)

    true_error_m = np.mean(true_error, axis=-1) * 100
    thresh_error_m = np.mean(error_thresh, axis=-1) * 100
    ddc_error_m = np.mean(error_ddc, axis=-1) * 100
    ddc_error_95_m = np.mean(error_ddc_95, axis=-1) * 100
    ddc_error_90_m = np.mean(error_ddc_90, axis=-1) * 100
    
    #results = pandas.DataFrame(index=epsilons)
    #results["True error"] = true_error_m
    #results["delta threshold"] = thresh_delta_m
    #results["epsilon threshold"] = thresh_eps_m
    #results["error threshold"] = thresh_error_m
    #results["delta DDC99"] = ddc_delta_m
    #results["epsilon DDC99"] = ddc_eps_m
    #results["error DDC99"] = ddc_error_m
    #results["delta DDC90"] = ddc_delta_90_m
    #results["epsilon DDC90"] = ddc_eps_90_m
    #results["error DDC90"] = ddc_error_90_m
    
    results = pandas.DataFrame(index=epsilons)
    results["delta threshold"] = thresh_delta_m
    results["std 1"] = thresh_delta_s
    results["epsilon threshold"] = thresh_eps_m
    results["std2"] = thresh_eps_s
    results["delta DDC99"] = ddc_delta_m
    results["std3"] = ddc_delta_s
    results["epsilon DDC99"] = ddc_eps_m
    results["std4"] = ddc_eps_s
    results["delta DDC90"] = ddc_delta_90_m
    results["std5"] = ddc_delta_90_s
    results["epsilon DDC90"] = ddc_eps_90_m
    results["std6"] = ddc_eps_90_s
    
    return(results)

In [11]:
thresholding_experiments("dirac", 4)

 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 5/6 [02:46<00:33, 33.23s/it]/Users/gregoirepacreau/opt/anaconda3/lib/python3.9/site-packages/statsmodels/robust/scale.py:297: RuntimeWarning: invalid value encountered in sqrt
  nscale = np.sqrt(scale_num / scale_denom)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:18<00:00, 33.04s/it]
/var/folders/m2/86ygsxzd7m79qj2clmkdp7840000gn/T/ipykernel_7875/1702517888.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  thresholdin

'\\begin{tabular}{lrrrrrrrrrrrr}\n\\toprule\n{} &  delta threshold &     std 1 &  epsilon threshold &  std2 &  delta DDC99 &      std3 &  epsilon DDC99 &      std4 &  delta DDC90 &      std5 &  epsilon DDC90 &      std6 \\\\\n\\midrule\n0.001 &        99.644294 &  0.022961 &                0.0 &   0.0 &    99.057677 &  0.029012 &       0.000000 &  0.000000 &    94.770355 &  0.053626 &       0.000000 &  0.000000 \\\\\n0.010 &        98.755192 &  0.026781 &                0.0 &   0.0 &    98.230978 &  0.036914 &       0.000000 &  0.000000 &    94.261670 &  0.102451 &       0.000000 &  0.000000 \\\\\n0.050 &        94.902280 &  0.013046 &                0.0 &   0.0 &    94.608613 &  0.017807 &       0.000000 &  0.000000 &    91.800374 &  0.060350 &       0.000000 &  0.000000 \\\\\n0.100 &        89.977695 &  0.004282 &                0.0 &   0.0 &    89.855158 &  0.016384 &       0.000000 &  0.000000 &    88.198693 &  0.109121 &       0.000000 &  0.000000 \\\\\n0.200 &        80.000000 & 

In [13]:
thresholding_experiments("gauss", 4)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:21<00:00, 33.59s/it]
/var/folders/m2/86ygsxzd7m79qj2clmkdp7840000gn/T/ipykernel_7875/543415301.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  thresholding_experiments("gauss", 4).to_latex()


'\\begin{tabular}{lrrrrrrrrrrrr}\n\\toprule\n{} &  delta threshold &     std 1 &  epsilon threshold &      std2 &  delta DDC99 &      std3 &  epsilon DDC99 &      std4 &  delta DDC90 &      std5 &  epsilon DDC90 &      std6 \\\\\n\\midrule\n0.001 &        99.623292 &  0.025332 &           0.034040 &  0.003335 &    99.045977 &  0.033346 &       0.055867 &  0.003126 &    94.786158 &  0.090941 &       0.053076 &  0.002851 \\\\\n0.010 &        98.757583 &  0.025223 &           0.371958 &  0.021653 &    98.164230 &  0.039929 &       0.596835 &  0.015386 &    94.097289 &  0.058224 &       0.565107 &  0.015959 \\\\\n0.050 &        94.874302 &  0.010575 &           1.867405 &  0.156868 &    94.455194 &  0.034646 &       3.006922 &  0.054897 &    91.128511 &  0.080560 &       2.841486 &  0.045882 \\\\\n0.100 &        89.947996 &  0.008220 &           3.993597 &  0.277455 &    89.623863 &  0.016597 &       6.185095 &  0.093035 &    87.109777 &  0.052272 &       5.802422 &  0.064097 \\\\\n0.200 &

# Abalone

In [14]:
def thresholding_experiments(option, intensity):
    # hamming distance between the masks
    hamming_thresh = np.zeros((len(epsilons), nrep))
    hamming_ddc = np.zeros((len(epsilons), nrep))
    hamming_ddc_95 = np.zeros((len(epsilons), nrep))
    hamming_ddc_90 = np.zeros((len(epsilons), nrep))

    #accuracy of the masks
    acc_thresh = np.zeros((len(epsilons), nrep))
    acc_ddc = np.zeros((len(epsilons), nrep))
    acc_ddc_95 = np.zeros((len(epsilons), nrep))
    acc_ddc_90 = np.zeros((len(epsilons), nrep))

    # estimated delta
    delta_thresh = np.zeros((len(epsilons), nrep))
    delta_ddc = np.zeros((len(epsilons), nrep))
    delta_ddc_95 = np.zeros((len(epsilons), nrep))
    delta_ddc_90 = np.zeros((len(epsilons), nrep))
    epsilon_thresh = np.zeros((len(epsilons), nrep))
    epsilon_ddc = np.zeros((len(epsilons), nrep))
    epsilon_ddc_95 = np.zeros((len(epsilons), nrep))
    epsilon_ddc_90 = np.zeros((len(epsilons), nrep))
    
    # spectral error of covariance
    true_error = np.zeros((len(epsilons), nrep))
    error_thresh = np.zeros((len(epsilons), nrep))
    error_ddc = np.zeros((len(epsilons), nrep))
    error_ddc_95 = np.zeros((len(epsilons), nrep))
    error_ddc_90 = np.zeros((len(epsilons), nrep))
    
    data = clean_abalone()
    sigma_true = np.cov(data.T)
    true_norm = np.linalg.norm(sigma_true, ord=2)

    for i,epsilon in enumerate(tqdm(epsilons)):
        for k in range(nrep):
            
            # generate data
            X_noisy, true_mask = contaminate_bernoulli(data, epsilon, intensity=intensity, option=option)
            
            # compute true error
            true_error[i,k] = np.linalg.norm(np.cov(X_noisy.T) - sigma_true, ord=2)/true_norm

            # find outliers
            # 1. find_perturbation
            thresh_mask, delta_thresh[i,k], epsilon_thresh[i,k] = find_stats_perturbation(X_noisy, true_mask, 3)
            delta_thresh[i,k] *= (1-epsilon) * 100
            epsilon_thresh[i,k] *= epsilon * 100
            
            # 2. find_outliers_DDC
            ddc_mask, _ = remove_outliers_DDC(X_noisy, 0.99)
            delta_ddc[i,k] = np.sum(ddc_mask*(1-true_mask))/np.sum(1-true_mask) * (1-epsilon) * 100
            epsilon_ddc[i,k] = np.sum(ddc_mask*true_mask)/np.sum(true_mask) * epsilon * 100
            
            # 3. find outliers DDC but with higher tolerance
            ddc_mask_95, _ = remove_outliers_DDC(X_noisy, 0.95)
            delta_ddc_95[i,k] = np.sum(ddc_mask_95*(1-true_mask))/np.sum(1-true_mask) * (1-epsilon) * 100
            epsilon_ddc_95[i,k] = np.sum(ddc_mask_95*true_mask)/np.sum(true_mask) * epsilon * 100
            
            # 4. find outliers DDC but with much higher tolerance
            ddc_mask_90, _ = remove_outliers_DDC(X_noisy, 0.9)
            delta_ddc_90[i,k] = np.sum(ddc_mask_90*(1-true_mask))/np.sum(1-true_mask) * (1-epsilon) * 100
            epsilon_ddc_90[i,k] = np.sum(ddc_mask_90*true_mask)/np.sum(true_mask) * epsilon * 100
            
            # 5. find estimation error under those filtration methods
            sigma_thresh = apply_estimator('oracleMV', X_noisy, mask=1-thresh_mask)
            error_thresh[i,k] = np.linalg.norm(sigma_true - sigma_thresh, ord=2)/true_norm
            sigma_ddc = apply_estimator('oracleMV', X_noisy, mask = 1-ddc_mask)
            error_ddc[i,k] = np.linalg.norm(sigma_true - sigma_ddc, ord=2)/true_norm
            sigma_ddc_95 = apply_estimator('oracleMV', X_noisy, mask=1-ddc_mask_95)
            error_ddc_95[i,k] = np.linalg.norm(sigma_true - sigma_ddc_95, ord=2)/true_norm
            sigma_ddc_90 = apply_estimator('oracleMV', X_noisy, mask=1-ddc_mask_90)
            error_ddc_90[i,k] = np.linalg.norm(sigma_true - sigma_ddc_90, ord=2)/true_norm

    thresh_delta_m = np.mean(delta_thresh, axis=-1) * (1-epsilon)
    thresh_delta_s = np.std(delta_thresh, axis=-1) 
    ddc_delta_m = np.mean(delta_ddc, axis=-1)
    ddc_delta_s = np.std(delta_ddc, axis=-1)
    ddc_delta_95_m = np.mean(delta_ddc_95, axis=-1)
    ddc_delta_95_s = np.std(delta_ddc_95, axis=-1)
    ddc_delta_90_m = np.mean(delta_ddc_90, axis=-1)
    ddc_delta_90_s = np.std(delta_ddc_90, axis=-1)

    thresh_eps_m = np.mean(epsilon_thresh, axis=-1)* epsilon
    thresh_eps_s = np.std(epsilon_thresh, axis=-1)
    ddc_eps_m = np.mean(epsilon_ddc, axis=-1)
    ddc_eps_s = np.std(epsilon_ddc, axis=-1)
    ddc_eps_95_m = np.mean(epsilon_ddc_95, axis=-1)
    ddc_eps_95_s = np.std(epsilon_ddc_95, axis=-1)
    ddc_eps_90_m = np.mean(epsilon_ddc_90, axis=-1)
    ddc_eps_90_s = np.std(epsilon_ddc_90, axis=-1)

    true_error_m = np.mean(true_error, axis=-1) * 100
    thresh_error_m = np.mean(error_thresh, axis=-1) * 100
    ddc_error_m = np.mean(error_ddc, axis=-1) * 100
    ddc_error_95_m = np.mean(error_ddc_95, axis=-1) * 100
    ddc_error_90_m = np.mean(error_ddc_90, axis=-1) * 100
    
    results = pandas.DataFrame(index=epsilons)
    results["delta threshold"] = thresh_delta_m
    results["std 1"] = thresh_delta_s
    results["epsilon threshold"] = thresh_eps_m
    results["std2"] = thresh_eps_s
    results["delta DDC99"] = ddc_delta_m
    results["std3"] = ddc_delta_s
    results["epsilon DDC99"] = ddc_eps_m
    results["std4"] = ddc_eps_s
    results["delta DDC90"] = ddc_delta_90_m
    results["std5"] = ddc_delta_90_s
    results["epsilon DDC90"] = ddc_eps_90_m
    results["std6"] = ddc_eps_90_s
    
    return(results)

In [17]:
thresholding_experiments("dirac", 4)

 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 5/6 [01:31<00:18, 18.38s/it]/Users/gregoirepacreau/opt/anaconda3/lib/python3.9/site-packages/statsmodels/robust/scale.py:297: RuntimeWarning: invalid value encountered in sqrt
  nscale = np.sqrt(scale_num / scale_denom)
/Users/gregoirepacreau/opt/anaconda3/lib/python3.9/site-packages/statsmodels/robust/scale.py:297: RuntimeWarning: invalid value encountered in sqrt
  nscale = np.sqrt(scale_num / scale_denom)
/Users/gregoirepacreau/opt/anaconda3/lib/python3.9/site-packages/statsmodels/robust/scale.py:297: RuntimeWarning: invalid value encountered in sqrt
  nscale = np.sqrt(scale_num / scale_denom)
/Users/gregoirepacreau/opt/anaconda3/lib/python3.9/site-packages/statsmodels/robust/scale.py:297: RuntimeWarning: invalid value encountered in sqrt
  nscale = np.sqrt(scale_num / scale_denom)
/Users/gregoirepacreau/opt/a

,delta threshold,std 1,epsilon threshold,std2,delta DDC99,std3,epsilon DDC99,std4,delta DDC90,std5,epsilon DDC90,std6
0.001,69.510658,0.000061,0.0,0.0,98.019977,0.012096,0.000000,0.000000,93.227836,0.012029,0.000000,0.000000
0.010,68.893075,0.004702,0.0,0.0,97.167720,0.034572,0.000000,0.000000,92.603435,0.030053,0.000000,0.000000
0.050,66.185157,0.028584,0.0,0.0,93.588426,0.030845,0.000000,0.000000,89.760656,0.078562,0.000000,0.000000
0.100,62.823492,0.014676,0.0,0.0,88.981781,0.039185,0.000000,0.000000,85.885730,0.059050,0.000000,0.000000
0.200,55.999761,0.001026,6.0,0.0,79.970908,0.030783,0.175691,0.190080,79.786776,0.209779,0.008544,0.014765
0.300,49.000000,0.000000,9.0,0.0,70.000000,0.000000,29.550225,0.058925,70.000000,0.000000,24.093514,0.237543


In [18]:
thresholding_experiments("gauss", 4)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:51<00:00, 18.58s/it]


,delta threshold,std 1,epsilon threshold,std2,delta DDC99,std3,epsilon DDC99,std4,delta DDC90,std5,epsilon DDC90,std6
0.001,69.511568,0.002238,0.015579,0.009150,98.021129,0.016365,0.054523,0.008109,93.248631,0.031935,0.052336,0.007657
0.010,68.893367,0.006470,0.165234,0.020827,97.165555,0.046433,0.559714,0.023987,92.618608,0.085088,0.537484,0.028638
0.050,66.168961,0.026857,0.853107,0.030987,93.488109,0.053311,2.834215,0.048427,89.730422,0.086845,2.711608,0.041950
0.100,62.776564,0.012697,1.776777,0.090727,88.786167,0.090075,5.754974,0.074845,85.812623,0.163853,5.478452,0.063130
0.200,55.939854,0.010943,3.976397,0.087040,79.563220,0.046659,12.527242,0.084296,77.690381,0.105061,11.631405,0.091140
0.300,48.990171,0.004170,6.603329,0.123053,68.002314,0.650338,21.391910,0.849766,66.904504,0.749545,19.547622,0.566308
